In [ ]:
def csv_df(url):
    chunksize = 50000
    df = None
    for dfchunk in pd.read_csv(url, chunksize=chunksize, header=0, iterator=True,
        usecols=["tpep_pickup_datetime", "tpep_dropoff_datetime", 
                 "passenger_count","trip_distance","PULocationID","DOLocationID",
                 "payment_type","fare_amount","total_amount"],
        parse_dates=["tpep_pickup_datetime","tpep_dropoff_datetime"]):
        if df is None:
            df = dfchunk.copy()
        else:
            df = df.append(dfchunk)
    # Merge the data with taxi zones table (taking only data we need for the QR#2)
    df_= df.merge(zonesLookup[['LocationID','Borough']],how='inner', left_on='PULocationID', right_on='LocationID')

    return(df_)


In [ ]:
def df_optimize(df):
    
    df = df[df.passenger_count>0]
    df = df[df.trip_distance>0]
    df = df[df.tpep_pickup_datetime<df.tpep_dropoff_datetime]
    df = df[df.tpep_pickup_datetime>datetime(2017,12,31)]
    df = df[df.fare_amount>0]
    
    data_int = df.select_dtypes(include=['int']).apply(pd.to_numeric,downcast='unsigned')
    data_float = df.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')
    data_obj = df.select_dtypes(include=['object']).astype('category')
    data_optimized = df.copy()
    data_optimized[data_int.columns] = data_int
    data_optimized[data_float.columns] = data_float
    data_optimized[data_obj.columns] = data_obj
    
    return(data_optimized)


In [ ]:

def rq1_process(df):
    # We create empty Dataframe, with two column, one for pick up time, and the other for the borough
    rq1_ = pd.DataFrame(columns=['PU_day','Borough'])
    # From pickup time we only use the day
    rq1_['PU_day'] = df['tpep_pickup_datetime'].apply(lambda x: pd.to_datetime(x).day)
    rq1_['Borough'] = df['Borough']
    return (rq1_)

In [ ]:
def rq1_plot_ny(df, days):
    # To show the daily trips in Jan (in NY), we group the rq1_jan by day values
    df[['PU_day','Borough']].groupby('PU_day').size()
    # and then we plot it
    fig, ax = plt.subplots(figsize=(15,5))
    # Modify the x-axis to show numbers between 1-31
    plt.xticks(list(range(1,days+1)))
    df[['PU_day','Borough']].groupby('PU_day').size().plot(ax=ax,kind="line")
    return


In [ ]:
def rq1_plot_boroughs(df):
    # Group by day and Borough and calculate the number of daily trips for each borough
    rq1_plot_ = df[['PU_day','Borough']].groupby(['PU_day','Borough'])['PU_day'].size().unstack()
    fig = plt.figure(figsize=(15,5))
    ax1 = fig.add_axes([0.1, 2.1, 0.8, 0.4])
    ax2 = fig.add_axes([0.1, 1.7, 0.8, 0.4])
    ax3 = fig.add_axes([0.1, 1.3, 0.8, 0.4])
    ax4 = fig.add_axes([0.1, 0.9, 0.8, 0.4])
    ax5 = fig.add_axes([0.1, 0.5, 0.8, 0.4])
    ax6 = fig.add_axes([0.1, 0.1, 0.8, 0.4])


    ax1.plot(rq1_plot_.iloc[:,0], label="Bronx")
    ax2.plot(rq1_plot_.iloc[:,1], label="Brooklyn")
    ax3.plot(rq1_plot_.iloc[:,2], label="EWR")
    ax4.plot(rq1_plot_.iloc[:,3], label="Manhattan")
    ax5.plot(rq1_plot_.iloc[:,4], label="Queens")
    ax6.plot(rq1_plot_.iloc[:,5], label="Staten Island")

    ax1.legend(loc="upper right")
    ax2.legend(loc="upper right")
    ax3.legend(loc="upper right")
    ax4.legend(loc="upper right")
    ax5.legend(loc="upper right")
    ax6.legend(loc="upper right")
    
    return(rq1_plot_.mean(axis=0))



In [ ]:
# the below function takes the data of one month and return the average of passengers by slots
def rq2_process(df):
    rq2_ = pd.DataFrame(columns=['PU_hour','passenger_count','Borough'])
    # From pickup time we only use the hour
    rq2_['PU_hour'] = df['tpep_pickup_datetime'].apply(lambda x: pd.to_datetime(x).hour)
    rq2_['passenger_count'] = df['passenger_count']
    rq2_['Borough'] = df['Borough']

    return (rq2_)

In [ ]:
def rq2_plot_ny(df):
    #df[['PU_hour','passenger_count']].groupby('PU_hour').sum()
    fig, ax = plt.subplots(figsize=(15,5))
    # Modify the x-axis to show numbers between 0-23
    plt.xticks(list(range(24)))
    df[['PU_hour','passenger_count']].groupby('PU_hour').sum().plot(ax=ax,kind="bar")
    return ()

In [ ]:
def rq2_plot_boroughs(df):
    # Modify the x-axis to show numbers between 0-23
    #plt.xticks(list(range(24)))
    
    # Group by day and Borough and calculate the number of daily trips for each borough
    rq2_plot_ = df[['PU_hour','passenger_count','Borough']].groupby(['PU_hour','Borough'])['PU_hour'].size().unstack()
    fig = plt.figure(figsize=(15,5))
    ax1 = fig.add_axes([0.1, 2.1, 0.8, 0.4])
    ax2 = fig.add_axes([0.1, 1.7, 0.8, 0.4])
    ax3 = fig.add_axes([0.1, 1.3, 0.8, 0.4])
    ax4 = fig.add_axes([0.1, 0.9, 0.8, 0.4])
    ax5 = fig.add_axes([0.1, 0.5, 0.8, 0.4])
    ax6 = fig.add_axes([0.1, 0.1, 0.8, 0.4])

    rq2_plot_.iloc[:,0].plot(ax=ax1,kind="bar", label="Bronx")
    rq2_plot_.iloc[:,1].plot(ax=ax2,kind="bar", label="Brookly")
    rq2_plot_.iloc[:,2].plot(ax=ax3,kind="bar", label="EWR")
    rq2_plot_.iloc[:,3].plot(ax=ax4,kind="bar", label="Manhattan")
    rq2_plot_.iloc[:,4].plot(ax=ax5,kind="bar", label="Queens")
    rq2_plot_.iloc[:,5].plot(ax=ax6,kind="bar", label="Staten Island")

    ax1.legend(loc="upper right")
    ax2.legend(loc="upper right")
    ax3.legend(loc="upper right")
    ax4.legend(loc="upper right")
    ax5.legend(loc="upper right")
    ax6.legend(loc="upper right")
    return ()

In [ ]:
def rq3_process(df):
    rq3_ = pd.DataFrame(columns=['Duration','Borough'])
    rq3_['Borough'] = df['Borough']
    start = df['tpep_pickup_datetime']
    #end   = data_jan_optimized.at[1,'tpep_dropoff_datetime']
    end = df['tpep_dropoff_datetime']

    delta = end-start
    rq3_['Duration'] = delta.dt.seconds/60
    return (rq3_)

In [ ]:
def rq3_polt_ny(df):
    # After running the first time we noticed very few entries with very high value in duration
    # so we are excluding them for the sake of getting better plot
    df=df[df.Duration<=60]
    fig, ax = plt.subplots(figsize=(20,7))
    plt.xticks(list(range(0,60,1)))
    sns.distplot(df['Duration'],hist=True, kde=False, 
                 bins=int(180/3), color = 'blue',
                 hist_kws={'edgecolor':'black'})
 

In [ ]:
def rq3_plot_boroughs(df):
    # After running the first time we noticed very few entries with very high value in duration
    # so we are excluding them for the sake of getting better plot
    df=df[df.Duration<=60]
    fig = plt.figure(figsize=(20,5))
    ax1 = fig.add_axes([0.1, 2.1, 0.8, 0.4])
    ax2 = fig.add_axes([0.1, 1.7, 0.8, 0.4])
    ax3 = fig.add_axes([0.1, 1.3, 0.8, 0.4])
    ax4 = fig.add_axes([0.1, 0.9, 0.8, 0.4])
    ax5 = fig.add_axes([0.1, 0.5, 0.8, 0.4])
    ax6 = fig.add_axes([0.1, 0.1, 0.8, 0.4])
    plt.xticks(list(range(0,60,1)))
    sns.distplot(df.loc[df['Borough'] == "Bronx"]['Duration'],hist=True, kde=False, 
             bins=int(180/3), color = 'blue', ax=ax1, label="Bronx",hist_kws={'edgecolor':'black'});
    sns.distplot(df.loc[df['Borough'] == "Brookly"]['Duration'],hist=True, kde=False, 
             bins=int(180/3), color = 'blue', ax=ax2, label="Brookly",hist_kws={'edgecolor':'black'});
    sns.distplot(df.loc[df['Borough'] == "EWR"]['Duration'],hist=True, kde=False, 
             bins=int(180/3), color = 'blue', ax=ax3, label="EWR",hist_kws={'edgecolor':'black'});
    sns.distplot(df.loc[df['Borough'] == "Manhattan"]['Duration'],hist=True, kde=False, 
             bins=int(180/3), color = 'blue', ax=ax4, label="Manhattan",hist_kws={'edgecolor':'black'});
    sns.distplot(df.loc[df['Borough'] == "Queens"]['Duration'],hist=True, kde=False, 
             bins=int(180/3), color = 'blue', ax=ax5, label="Queens",hist_kws={'edgecolor':'black'});
    sns.distplot(df.loc[df['Borough'] == "Staten Island"]['Duration'],hist=True, kde=False, 
             bins=int(180/3), color = 'blue', ax=ax6, label="Staten Island",hist_kws={'edgecolor':'black'});


    ax1.legend(loc="upper right")
    ax2.legend(loc="upper right")
    ax3.legend(loc="upper right")
    ax4.legend(loc="upper right")
    ax5.legend(loc="upper right")
    ax6.legend(loc="upper right")
    return ()


In [ ]:
def rq4_process(df):
    rq4_= df[['payment_type','Borough']].groupby(['payment_type','Borough']).size().unstack()
    # we change the index from integer to the name of the method
    rq4_.index = ['Credit Card','Cash', 'No Charge', 'Dispute']
    # drop the row of 'No Charge', since we don't need to analyze
    rq4_ = rq4_.drop('No Charge')
    # drop the column of 'Unkown'
    rq4_ = rq4_.drop('Unknown', 1)
    
    return (rq4_)

In [ ]:
def rq4_plot(df):
    fig = plt.figure(constrained_layout=True,figsize=(15, 8))
    spec = gridspec.GridSpec(ncols=3, nrows=2, figure=fig)
    ax1 = fig.add_subplot(spec[0, 0])
    ax2 = fig.add_subplot(spec[0, 1])
    ax3 = fig.add_subplot(spec[0, 2])
    ax4 = fig.add_subplot(spec[1, 0])
    ax5 = fig.add_subplot(spec[1, 1])
    ax6 = fig.add_subplot(spec[1, 2])
    
    df.iloc[:,0].plot(ax=ax1, kind='bar', label='Bronx', rot=0)
    df.iloc[:,1].plot(ax=ax2, kind='bar', label='Brooklyn', rot=0)
    df.iloc[:,2].plot(ax=ax3, kind='bar', label='EWR', rot=0)
    df.iloc[:,3].plot(ax=ax4, kind='bar', label='Manhattan', rot=0)
    df.iloc[:,4].plot(ax=ax5, kind='bar', label='Queens', rot=0)
    df.iloc[:,5].plot(ax=ax6, kind='bar', label='Staten Island', rot=0)
    
    ax1.legend(loc="upper right")
    ax2.legend(loc="upper right")
    ax3.legend(loc="upper right")
    ax4.legend(loc="upper right")
    ax5.legend(loc="upper right")
    ax6.legend(loc="upper right")
    
    return()

In [ ]:
def rq5_process(df):
    
    
    rq5_ = pd.DataFrame(columns=['Duration','Distance'])
    rq5_['Distance'] = df['trip_distance']
    start = df['tpep_pickup_datetime']
    end = df['tpep_dropoff_datetime']

    delta = end-start
    rq5_['Duration'] = delta.dt.seconds/60
    
    #rq5_ = rq5_[rq5_.Duration<60].sample(1000)

    # we observed some duration values = 0, and some negative Distance, we cleanse the data: 
    rq5_ = rq5_[rq5_.Duration>0]
    rq5_ = rq5_[rq5_.Distance>0]
    
    rq5_=rq5_.sort_values('Distance')
    
    return(rq5_)


In [ ]:
def rq5_plot(df):
    fig, ax = plt.subplots(figsize=(20,7))

    df=df.sort_values('Distance')
    
    x = np.arange(1000)
    plt.plot(x, df['Distance'])
    plt.plot(x, df['Duration'])
    return()


In [ ]:
def rq5_plot_disc(df):
    fig, ax = plt.subplots(figsize=(20,7))
    df.plot(ax=ax, kind='line')

    return()


In [ ]:
def rq5_process_sample(df):
    
    
    rq5_ = pd.DataFrame(columns=['Duration','Distance'])
    rq5_['Distance'] = df['trip_distance']
    start = df['tpep_pickup_datetime']
    end = df['tpep_dropoff_datetime']

    delta = end-start
    rq5_['Duration'] = delta.dt.seconds/60
    
    rq5_sample = rq5_[rq5_.Duration<60].sample(1000)

    # we observed some duration values = 0, and some negative Distance, we cleanse the data: 
    rq5_sample = rq5_sample[rq5_sample.Duration>0]
    rq5_sample = rq5_sample[rq5_sample.Distance>0]
    
    rq5_sample=rq5_sample.sort_values('Distance')
    
    return(rq5_sample)


In [ ]:
def rq5_process_discrete(df):
    
    
    rq5_ = pd.DataFrame(columns=['Duration','Distance'])
    # we convert distance to discrete values in order to group them and take average of duration
    rq5_['Distance'] = df['trip_distance'].apply(lambda x: int(x/10))
    
    start = df['tpep_pickup_datetime']
    end = df['tpep_dropoff_datetime']

    delta = end-start
    rq5_['Duration'] = delta.dt.seconds/60
    
    #rq5_sample = rq5_[rq5_.Duration<60].sample(1000)

    # we observed some duration values = 0, and some negative Distance, we cleanse the data: 
    rq5_ = rq5_[rq5_.Duration>0]
    rq5_ = rq5_[rq5_.Distance>0]
    
    rq5_disc = rq5_[['Distance','Duration']].groupby('Distance').mean()
    
    rq5_disc=rq5_disc.sort_values('Distance')
    
    return(rq5_disc)

In [ ]:
def do_PairTest(b1,b2):
    b_1 = cq1_data['price_per_mile'][cq1_data.Borough==b1]
    b_2 = cq1_data['price_per_mile'][cq1_data.Borough==b2]
    m_ = min(len(b_1),len(b_2))
    b_s, b_p = ttest_ind(b_1[:m_],b_2[:m_])

    if b_p > alpha:
        print('H0 cannot be rejected, Same distribution between {0} and {1} with stats={2:.3f} p-value = {3:.3f}.'.format(b1,b2,b_s,b_p))
    else:
        print('H0 rejected, Different distribution between {0} and {1} with stats={2:.3f} p-value = {3:.3f}.'.format(b1,b2,b_s,b_p))
    